In [33]:
import math
import datetime

import pandas as pd
import numpy as np
import folium
import plotly
import plotly.express as px
import plotly.graph_objects as go

In [34]:
def convert_timestring_to_time(date_string):
    if date_string is np.nan:
        return date_string
    idx = date_string.find('M')
    if( idx ==-1):
        return datetime.datetime.strptime(date_string, '%H:%M')
    idx -=2
    if date_string[idx] == ' ':
        return datetime.datetime.strptime(date_string, '%I:%M %p')
    return datetime.datetime.strptime(date_string, '%I:%M%p')

In [35]:
df = pd.read_csv("../../data/school_shooting_data.csv")
df['time'] = df['time'].apply(lambda x: convert_timestring_to_time(x))
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')

In [36]:
df = df[df['long'].notna()]
df.reset_index(drop=True, inplace=True)

In [37]:
def generate_color(killed, injured):
    if killed ==0 and injured==0:
        return 'blue'
    if killed ==0 :
        return 'gray'
    return 'crimson'

def calculate_circle_radius(max_affected, current_affected):
    return math.log(current_affected/ max_affected * 100 + 1.) * 2 + 4

In [38]:
max_killed_and_injured = max(df['killed']) + max(df['injured'])
# Make an empty map
m = folium.Map(location=[40,-100], tiles="OpenStreetMap", zoom_start=4.5)
for i in range(0,df.shape[0]):
   killed_and_injured = df.iloc[i]['killed'] + df.iloc[i]['injured']
   radius = calculate_circle_radius(max_killed_and_injured, killed_and_injured)
   folium.CircleMarker(
      location=[df.iloc[i]['lat'], df.iloc[i]['long']],
      popup=df.iloc[i]['school_name'],
      radius=radius,
      color=generate_color(df.iloc[i]['killed'], df.iloc[i]['injured']),
      fill=True,
      fill_color=generate_color(df.iloc[i]['killed'], df.iloc[i]['injured'])
   ).add_to(m)

# Show the map
m


In [39]:
def plot_student_race_distribution(races, column_names, school_name, file_path):
    races = races.fillna(0)
    fig = px.pie(values=races.values[0], names=column_names, title='Population of '+school_name, width=700)
    fig.update_layout(legend_title="Ethnicity")
    fig.update_traces(hovertemplate='Ethnicity: %{label} <br>  Count: %{value}')
    plotly.offline.plot(fig, filename=file_path)

In [40]:
df.tail()
df.reset_index(drop=True, inplace=True)

In [41]:
column_names = ['White', 'Black', 'Hispanic', 'Asian',
       'American/Indian/Alaska native', 'Hawaiian native/Pacific islander']
df['ethnicity_diagram_path'] = np.nan
for i in range(0,df.shape[0]):
    races = df.loc[[i], ['white', 'black', 'hispanic', 'asian',
       'american_indian_alaska_native', 'hawaiian_native_pacific_islander']]
    races = races._convert(numeric=True)
    schoolname = df.iloc[i]['school_name']
    filepath = './ethinicity_diagrams/' + schoolname.replace('/', '_') +'_ethinicity.html'
    filepathtosave= '../docs/htmls/ethinicity_diagrams/' + schoolname.replace('/', '_') +'_ethinicity.html'
    plot_student_race_distribution(races, column_names, df.iloc[i]['school_name'], filepathtosave)
    df.at[i,'ethnicity_diagram_path']= filepath

AttributeError: 'DataFrame' object has no attribute '_convert'

In [ ]:
max_killed_and_injured = max(df['killed']) + max(df['injured'])
# Make an empty map
m = folium.Map(location=[40,-100], tiles="OpenStreetMap", zoom_start=4.5)
for i in range(0,df.shape[0]):
   killed_and_injured = df.iloc[i]['killed'] + df.iloc[i]['injured']
   radius = calculate_circle_radius(max_killed_and_injured, killed_and_injured)
   html="""
    <iframe src=\"""" + df['ethnicity_diagram_path'][i] + """\" width="850" height="400"  frameborder="0">    
    """
   diagram = folium.Popup(folium.Html(html, script=True))
   folium.CircleMarker(
      location=[df.iloc[i]['lat'], df.iloc[i]['long']],
      radius=radius,
      popup=diagram,
      color=generate_color(df.iloc[i]['killed'], df.iloc[i]['injured']),
      fill=True,
      fill_color=generate_color(df.iloc[i]['killed'], df.iloc[i]['injured'])
   ).add_to(m)

# Show the map
# m
m.save('../docs/assets/htmls/shootings_details.html')


KeyError: 'ethnicity_diagram_path'